## **Aplicação de Knn e seleção de top12 bases de dados por F1Score **

In [ ]:
# Importando as libs
import os
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix

datafiles = [
    '../Aula11/hogfeat_128_16_2_9_pca.csv.gz',
    '../Aula11/hogfeat_128_32_2_9.csv.gz',
    '../Aula11/hogfeat_256_32_2_9.csv.gz',
    '../Aula11/hogfeat_256_64_2_9.csv.gz',
    '../Aula11/lbpfeat_256_12_96.csv.gz',
    '../Aula11/hogfeat_128_16_2_9.csv.gz',
    '../Aula11/hogfeat_128_8_2_9_pca.csv.gz',
    '../Aula11/hogfeat_256_4_2_9_pca.csv.gz',
    '../Aula11/hogfeat_256_8_2_9_pca.csv.gz',
    '../Aula11/lbpfeat_256_24_192.csv.gz',
    '../Aula11/hogfeat_128_16_4_9_pca.csv.gz',
    '../Aula11/hogfeat_128_8_2_9.csv.gz',
    '../Aula11/hogfeat_256_4_2_9.csv.gz',
    '../Aula11/hogfeat_256_8_2_9.csv.gz',
    '../Aula11/lbpfeat_256_3_24.csv.gz',
    '../Aula11/hogfeat_128_16_4_9.csv.gz',
    '../Aula11/hogfeat_256_32_2_9_pca.csv.gz',
    '../Aula11/hogfeat_256_64_2_18.csv.gz',
    '../Aula11/lbpfeat_256_1_8.csv.gz',
    '../Aula11/lbpfeat_256_6_48.csv.gz'
]



## Lendo dataframes do disco

In [4]:
# Loop que percorre todos os datafiles, carregando cada DataFrame e armazenando em um dicionário.
dfs = {}
for filepath in datafiles:
    # Extrai o nome do arquivo sem extensão .csv.gz
    filename = os.path.basename(filepath)
    key = filename.replace('.csv.gz', '')
    # Imprime o arquivo que está sendo lido
    print(f"Lendo arquivo: {filename}")
    # Carrega o DataFrame
    df = pd.read_csv(filepath)
    dfs[key] = df

Lendo arquivo: hogfeat_128_16_2_9_pca.csv.gz
Lendo arquivo: hogfeat_128_32_2_9.csv.gz
Lendo arquivo: hogfeat_256_32_2_9.csv.gz
Lendo arquivo: hogfeat_256_64_2_9.csv.gz
Lendo arquivo: lbpfeat_256_12_96.csv.gz
Lendo arquivo: hogfeat_128_16_2_9.csv.gz
Lendo arquivo: hogfeat_128_8_2_9_pca.csv.gz
Lendo arquivo: hogfeat_256_4_2_9_pca.csv.gz
Lendo arquivo: hogfeat_256_8_2_9_pca.csv.gz
Lendo arquivo: lbpfeat_256_24_192.csv.gz
Lendo arquivo: hogfeat_128_16_4_9_pca.csv.gz
Lendo arquivo: hogfeat_128_8_2_9.csv.gz
Lendo arquivo: hogfeat_256_4_2_9.csv.gz
Lendo arquivo: hogfeat_256_8_2_9.csv.gz
Lendo arquivo: lbpfeat_256_3_24.csv.gz
Lendo arquivo: hogfeat_128_16_4_9.csv.gz
Lendo arquivo: hogfeat_256_32_2_9_pca.csv.gz
Lendo arquivo: hogfeat_256_64_2_18.csv.gz
Lendo arquivo: lbpfeat_256_1_8.csv.gz
Lendo arquivo: lbpfeat_256_6_48.csv.gz


## Loop de processamento de cada dataframe

In [11]:
# Para cada DataFrame em dfs, realiza holdout e KFold para K de 1 a 10, calculando o F1 Score
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

# Dicionário para armazenar os resultados finais
tabela_resultados = []
distance_metric = 'euclidean'
# distance_metric = 'manhattan'
# distance_metric = 'chebyshev'
# Função para mapear raças para espécies
def raca_para_especie(raca):
    if raca in ['basset_hound', 'saint_bernard']:
        return 'dog'
    elif raca in ['Birman', 'Persian']:
        return 'cat'
    else:
        return raca  # fallback

for nome, df in dfs.items():
    # Remove a penúltima coluna (nome da imagem)
    X = df.iloc[:, :-2]
    # A última coluna é a raça (target) - converte para espécie
    y = df.iloc[:, -1].apply(raca_para_especie)
    print(f"Processando dataset: {nome} Shape: {X.shape} Classes: {y.unique()}")
    print()
    
    resultados_holdout = []
    resultados_kfold = []
    ks = list(range(1, 11))
    
    for k in ks:
        # Holdout: 10% para teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
        knn = KNeighborsClassifier(n_neighbors=k, metric=distance_metric)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        f1_holdout = f1_score(y_test, y_pred, average='weighted')
        resultados_holdout.append(f1_holdout)
        
        # KFold: 10 folds usando cross_val_score
        f1_kfold_scores = cross_val_score(
            KNeighborsClassifier(n_neighbors=k, metric=distance_metric), 
            X, y, 
            cv=10, 
            scoring='f1_weighted'
        )
        f1_kfold_mean = f1_kfold_scores.mean()
        resultados_kfold.append(f1_kfold_mean)
    
    # Monta a tabela para o dataset atual
    df_result = pd.DataFrame({
        'K': ks,
        'Holdout': resultados_holdout,
        'KFold': resultados_kfold
    })
    df_result['Dataset'] = nome
    tabela_resultados.append(df_result)
    
    print(f"Resultados para o dataset: {nome}")
    print(df_result[['K', 'Holdout', 'KFold']])
    print("-" * 50)

# Junta todos os resultados em uma única tabela
tabela_final = pd.concat(tabela_resultados, ignore_index=True)


Processando dataset: hogfeat_128_16_2_9_pca Shape: (800, 117) Classes: ['dog' 'cat']

Resultados para o dataset: hogfeat_128_16_2_9_pca
    K   Holdout     KFold
0   1  0.696970  0.718340
1   2  0.699812  0.708778
2   3  0.718670  0.692248
3   4  0.700000  0.728183
4   5  0.727935  0.693962
5   6  0.723444  0.727165
6   7  0.702024  0.674498
7   8  0.698303  0.711183
8   9  0.653333  0.663408
9  10  0.744246  0.705534
--------------------------------------------------
Processando dataset: hogfeat_128_32_2_9 Shape: (800, 324) Classes: ['dog' 'cat']

Resultados para o dataset: hogfeat_128_32_2_9
    K   Holdout     KFold
0   1  0.685089  0.695902
1   2  0.674185  0.701429
2   3  0.627668  0.700525
3   4  0.687060  0.741569
4   5  0.567992  0.711904
5   6  0.696970  0.756047
6   7  0.605003  0.716456
7   8  0.708815  0.756524
8   9  0.672721  0.723904
9  10  0.681478  0.750599
--------------------------------------------------
Processando dataset: hogfeat_256_32_2_9 Shape: (800, 1764) Cla

In [ ]:
# Função para extrair características do nome do arquivo
def extrair_caracteristicas(nome_arquivo):
    if 'hogfeat' in nome_arquivo:
        # Formato: hogfeat_128_16_2_9_pca
        partes = nome_arquivo.split('_')
        resolucao = partes[1]
        pixels_per_cell = partes[2]
        cells_per_block = partes[3]
        orientation = partes[4]
        pca = 'PCA' if 'pca' in nome_arquivo else 'Sem PCA'
        return f"{resolucao}x{pixels_per_cell}x{cells_per_block}x{orientation}", pca
    elif 'lbpfeat' in nome_arquivo:
        # Formato: lbpfeat_256_12_96
        partes = nome_arquivo.split('_')
        resolucao = partes[1]
        raio = partes[2]
        pontos = partes[3]
        return f"{resolucao}x{raio}x{pontos}", "LBP"
    else:
        return "Desconhecido", "Desconhecido"

# Cria uma lista para armazenar as linhas no novo formato
linhas = []

# Para cada dataset, cria uma linha conforme o formato desejado
for nome in dfs.keys():
    df_ds = tabela_final[tabela_final['Dataset'] == nome]
    
    # Extrai os valores de Holdout e KFold
    holdout = df_ds[df_ds['Tipo'] == 'Holdout'].iloc[0, 2:].values
    kfold = df_ds[df_ds['Tipo'] == 'KFold'].iloc[0, 2:].values
    
    # Obtém o shape do dataframe original
    atributos = dfs[nome].shape[1] - 2  # Remove nome da imagem e target
    
    # Extrai características do nome do arquivo
    resolucao_detalhes, pca_info = extrair_caracteristicas(nome)
    
    # Monta a linha conforme o formato desejado
    linha = [nome, atributos, resolucao_detalhes, pca_info] + list(holdout) + list(kfold)
    linhas.append(linha)

# Monta o cabeçalho conforme o exemplo do enunciado
ks = list(range(1, 11))
cabecalho = ['Dataset', 'Atributos', 'Resolução/Raio/Pontos', 'Resolução/Pixels per Cell/Cells per Block/Orientation/PCA']
cabecalho += [f'K={k}' for k in ks] * 2  # Holdout e KFold

# Cria o DataFrame final
df_final = pd.DataFrame(linhas, columns=cabecalho)

# Salva no formato final para Excel
df_final.to_csv('tabela_f1_scores_formatada_para_excel.csv', index=False, sep=';')
print("Tabela salva em 'tabela_f1_scores_formatada_para_excel.csv' no formato de colunas conforme solicitado.")
print(f"Total de datasets processados: {len(linhas)}")


In [13]:

# Reorganiza para o formato solicitado: uma linha para o K, uma para holdout, outra para kfold, por dataset
tabelas_formatadas = []
for nome in dfs.keys():
    df_ds = tabela_final[tabela_final['Dataset'] == nome]
    # linha_k = ['K'] + list(df_ds['K'])
    linha_holdout = ['Holdout'] + list(df_ds['Holdout'])
    linha_kfold = ['KFold'] + list(df_ds['KFold'])
    tabela_ds = pd.DataFrame([linha_holdout, linha_kfold])
    tabela_ds.columns = ['Tipo'] + [f'K={k}' for k in df_ds['K']]
    tabela_ds.insert(0, 'Dataset', nome)
    tabelas_formatadas.append(tabela_ds)

# Exibe a tabela final formatada
tabela_exibicao = pd.concat(tabelas_formatadas, ignore_index=True)
print("Tabela final de F1 Score para todos os datasets:")
print(tabela_exibicao)



Tabela final de F1 Score para todos os datasets:
                   Dataset     Tipo       K=1       K=2       K=3       K=4  \
0   hogfeat_128_16_2_9_pca  Holdout  0.696970  0.699812  0.718670  0.700000   
1   hogfeat_128_16_2_9_pca    KFold  0.718340  0.708778  0.692248  0.728183   
2       hogfeat_128_32_2_9  Holdout  0.685089  0.674185  0.627668  0.687060   
3       hogfeat_128_32_2_9    KFold  0.695902  0.701429  0.700525  0.741569   
4       hogfeat_256_32_2_9  Holdout  0.616368  0.622879  0.615830  0.681478   
5       hogfeat_256_32_2_9    KFold  0.657045  0.661237  0.639467  0.683531   
6       hogfeat_256_64_2_9  Holdout  0.679024  0.687060  0.672721  0.722222   
7       hogfeat_256_64_2_9    KFold  0.720480  0.726021  0.728927  0.740585   
8        lbpfeat_256_12_96  Holdout  0.637443  0.650202  0.662447  0.667519   
9        lbpfeat_256_12_96    KFold  0.671453  0.648015  0.689599  0.678972   
10      hogfeat_128_16_2_9  Holdout  0.658174  0.658174  0.696970  0.650000   
11 